In [ ]:
#@title <Center>CPU CHECK</Center>
!lscpu

In [ ]:
#@title <Center>CPU CHECK</Center>
import torch

if torch.cuda.is_available():
    print("GPU Found:", torch.cuda.get_device_name(0))
    print("Memory Allocated:", torch.cuda.memory_allocated(0))
else:
    print("No GPU found.")


In [ ]:
#@title <Center> Mount Google Drive </Center>
#@markdown <Center> <img src="https://kstatic.googleusercontent.com/files/e20dbc1db7b6eab22fb2d07df8043f8a74b301dbb8c176e6a1992b2dc229c5ae94b5c260cf549c41712fa40b7e639166c6a8461303a27c2a3b9c55b8ca155ec3" class="icon" alt="Icon">
#@markdown <h6> Made In Ani Animesh </h6>
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title <Center><font size="5">Install FFmpeg 7.0.1</font></Center> { vertical-output: true }
# ==========================
# Install Latest FFmpeg in Colab (v6.1.3)
# ==========================
from IPython.display import clear_output
import os, urllib.request
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/yunooooo/gcct/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    loadingAn,
    textAn,
)

loadingAn(name="lds")
textAn("Installing FFmpge [Ani]...", ty='twg')

# Remove old FFmpeg if exists
os.system('apt-get remove ffmpeg -y')

# Download latest FFmpeg static build
os.system('wget https://johnvansickle.com/ffmpeg/releases/ffmpeg-release-amd64-static.tar.xz -O ffmpeg.tar.xz')

# Extract
os.system('tar -xvf ffmpeg.tar.xz')

# Find extracted folder
ffmpeg_folder = [f for f in os.listdir() if f.startswith('ffmpeg') and os.path.isdir(f)][0]

# Move binaries to /usr/local/bin
os.system(f'mv {ffmpeg_folder}/ffmpeg /usr/local/bin/')
os.system(f'mv {ffmpeg_folder}/ffprobe /usr/local/bin/')

# Make executable
os.system('chmod +x /usr/local/bin/ffmpeg')
os.system('chmod +x /usr/local/bin/ffprobe')

# Verify installation
clear_output()
print('Installation finished.')
!ffmpeg -version


In [ ]:
#@title <font size="5">← ឵឵<i>Install FFmpeg Progress Bar</font> { vertical-output: true }
from IPython.display import clear_output
import os, urllib.request
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/yunooooo/gcct/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    loadingAn,
    textAn,
)

loadingAn(name="lds")
textAn("Installing FFmpge [Ani]...", ty='twg')
os.system('pip install https://github.com/althonos/ffpb/archive/master.zip')
clear_output()
print('Installation finished.')
!ffpb -version

In [ ]:
#@title 🎬 Anime Encoder (x264/x265/NVENC) 10-bit + Auto Folder + Live Progress
import os, subprocess, re, time
from pathlib import Path

# ---------- Colors for Progress ----------
RESET = "\033[0m"
RED = "\033[91m"
GREEN = "\033[92m"
YELLOW = "\033[93m"
CYAN = "\033[96m"

# ==========================
# User Parameters
# ==========================
input_folder = ''  #@param {type:"string"}
output_folder = ''  #@param {type:"string"}
encoder = "x265"  #@param ["x264","x265","h264_nvenc","hevc_nvenc"]
resolution = "720p"  #@param ["720p","1080p"]
crf_cq = "21"  #@param {type:"string"} # CRF for CPU, CQ for NVENC
audio_map = ['0:1','0:2']  #@param
output_file_type = "mkv"  #@param ["mkv","mp4"]
my_suffixes = (".mp4",".mov",".mkv",".avi",".ts",".flv",".webm",".wmv",".mpg",".m4v",".f4v",".m2ts",".mpeg",".3gp")

Path(output_folder).mkdir(parents=True, exist_ok=True)

# ==========================
# FFmpeg live progress function
# ==========================
def run_ffmpeg_progress(cmd, output_path, total_size_est=0):
    """Run FFmpeg and show advanced live in-place progress bar"""
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
    duration = None
    pattern_duration = re.compile(r"Duration: (\d+):(\d+):([\d.]+)")
    pattern_time = re.compile(r"time=(\d+):(\d+):([\d.]+)")
    start_time = time.time()

    while True:
        line = process.stdout.readline()
        if not line:
            break
        line = line.strip()

        if duration is None:
            dur_match = pattern_duration.search(line)
            if dur_match:
                h, m, s = map(float, dur_match.groups())
                duration = h*3600 + m*60 + s

        if duration:
            time_match = pattern_time.search(line)
            if time_match:
                h, m, s = map(float, time_match.groups())
                elapsed = h*3600 + m*60 + s
                pct = (elapsed/duration)*100
                elapsed_time = time.time() - start_time

                # Current size
                size_bytes = os.path.getsize(output_path) if os.path.exists(output_path) else 0
                current_size = size_bytes / (1024*1024)
                kbps = (size_bytes / elapsed_time / 1024) if elapsed_time > 0 else 0

                # ETA
                remaining = ((100 - pct) * (elapsed_time / pct)) if pct > 0 else 0
                eta_h, rem = divmod(int(remaining), 3600)
                eta_m, eta_s = divmod(rem, 60)

                # Progress bar
                bar_len = 25
                filled = int(bar_len * pct / 100)
                bar = GREEN + "█" * filled + RESET + "░" * (bar_len - filled)

                # Print in-place
                print(
                    f"\r{CYAN}‣ {RESET}Progress: {YELLOW}{pct:.2f}%{RESET} {bar} | "
                    f"{CYAN}Size:{RESET} {current_size:.2f} MB | "
                    f"{CYAN}Speed:{RESET} {kbps:.2f} KB/s | "
                    f"{CYAN}Time Took:{RESET} {int(elapsed_time//60)}m {int(elapsed_time%60)}s | "
                    f"{CYAN}Time Left:{RESET} {eta_h}h {eta_m}m {eta_s}s",
                    end='', flush=True
                )

    print()  # New line after completion
    process.wait()

# ==========================
# Encoding Loop
# ==========================
for filename in os.listdir(input_folder):
    if filename.endswith(my_suffixes):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, f"{filename.rpartition('.')[0]}.{output_file_type}")

        cmd = ["ffmpeg", "-stats", "-i", input_path]

        # -------------------------
        # Video Encoder
        # -------------------------
        if encoder == "x264":
            cmd += ["-c:v","libx264","-preset","slower","-tune","animation","-crf",crf_cq,"-pix_fmt","yuv420p"]
            cmd += ["-x264-params","me=umh:ref=6:deblock=1,1,1:subme=7:psy=1:psy_rd=0.40:bframes=5:b_pyramid=2:b_adapt=2"]
        elif encoder == "x265":
            cmd += ["-c:v","libx265","-preset","medium","-tune","animation","-crf",crf_cq,"-pix_fmt","yuv420p10le"]
            cmd += ["-x265-params","bframes=6:ref=4:aq-mode=3:aq-strength=1.0:psy-rd=1.0:deblock=1,1:rd=4"]
        elif encoder == "h264_nvenc":
            cmd += ["-c:v","h264_nvenc","-preset","slow","-cq",crf_cq,"-pix_fmt","yuv420p"]
        elif encoder == "hevc_nvenc":
            cmd += ["-c:v","hevc_nvenc","-preset","p7","-cq",crf_cq,"-pix_fmt","p010le","-tune","hq"]

        # -------------------------
        # Filters
        # -------------------------
        vf_list = []
        if resolution=="720p":
            vf_list.append("scale=1280:720")
        elif resolution=="1080p":
            vf_list.append("scale=1920:1080")
        vf_list.append("smartblur=1.5:-0.35:-3.5:0.65:0.25:2.0")
        vf_list.append("drawtext=fontfile=/content/drive/MyDrive/Font/A.ttf:text='Encoded By @Ani_Animesh':x=1040:y=650:fontsize=20:fontcolor=white:enable='between(t,200,400)'")
        vf_list.append("drawtext=fontfile=/content/drive/MyDrive/Font/cream-DEMO.ttf:text='www.AnimeToki.com':x=10:y=10:fontsize=20:fontcolor=red:enable='between(t,600,630)'")
        cmd += ["-vf", ",".join(vf_list)]

        # -------------------------
        # Map all audio & subtitles
        # -------------------------
        cmd += ["-map","0:v:0"]
        for audio in audio_map:
            cmd += ["-map", audio]
        cmd += ["-map","0:s?"]

        # -------------------------
        # Audio codec
        # -------------------------
        for idx, audio in enumerate(audio_map):
            if encoder in ["x264","x265"]:
                cmd += ["-c:a:"+str(idx),"libopus","-b:a:"+str(idx),"96k"]
            else:
                cmd += ["-c:a:"+str(idx),"aac","-b:a:"+str(idx),"96k"]

        # -------------------------
        # Metadata
        # -------------------------
        cmd += [
            "-metadata","title=Episode Unknow xD",
            "-metadata","author=AnimeToki.com",
            "-metadata","copyright=Copyright 2025 Me",
            "-metadata:s:0","title=Presented By AnimeToki",
            "-metadata:s:a:0","title=Ani Animesh",
            "-metadata:s:a:1","title=AnimeToki.com",
            "-metadata:s:s:0","title=AnimeToki",
            "-metadata:s:s:1","title=@Ani_Animesh",
            output_path
        ]

        # -------------------------
        # Run FFmpeg with live progress
        # -------------------------
        run_ffmpeg_progress(cmd, output_path)


In [ ]:
#@markdown <h3><b>FFMPEG Encode x265 10bit :</h3>

import os, sys, re

input_folder= '' #@param {type:"string"}
output_folder= '' #@param {type:"string"}
#@markdown <h3><b>Choose your encode setting :</h5>
#@markdown <h><i>main = default | main10 = HEVC 10 Bit</i>
encode_setting = "X265 AAC 720p" #@param ["X265 AAC 720p", "1080p sub", "X265 Copy", "X265 AAC 1080p", "1080p dub", "720p sub", "720p dub"]
x265_setting = "frame-threads=4:bframes=8:psy-rd=1:aq-mode=3:aq-strength=0.8:deblock=1,1" #@param ["frame-threads=4:bframes=8:psy-rd=1:aq-mode=3:aq-strength=0.8:deblock=1,1", "frame-threads=4:bframes=8:psy-rd=1.5:psy-rdoq=2:aq-mode=3"]
output_file_type = 'mkv' #@param ["mkv", "mp4"]
#@markdown <h3><b>Video quality :</h3>
#@markdown <h><i>0 (Best) to 51 (Worst)</i><h>
set_crf ='21.2'#@param {type:"string"}
set_audio ='0:1' #@param {type:"string"}
set_audio1 ='0:2' #@param {type:"string"}
my_suffixes = (".mp4", ".mov", ".mkv", ".avi", ".ts", ".flv", ".webm", ".wmv", ".mpg", ".m4v", ".f4v", ".m2ts", ".mpeg", ".3gp", ".MP4", ".MOV", ".MKV", ".AVI", ".TS", ".FLV", ".WEBM", ".WMV", ".MPG", ".M4V", ".F4V", ".M2TS", ".MPEG", ".3GP")

from pathlib import Path
Path(output_folder).mkdir(parents=True, exist_ok=True)

if encode_setting == 'X265 AAC 720p':
 for filename in os.listdir(input_folder):
     if (filename.endswith(my_suffixes)):
             cmd = !ffpb -stats  -i "$input_folder/{filename}" -metadata author="AnimeSakura.co" -metadata encoded_by="Ani Animesh" -metadata copyright="Copyright 2023 Me" -metadata comment="X265-CRF$set_crf-AAC" -c:v libx265 -color_primaries 1 -color_range 1 -color_trc 1 -colorspace 1 -profile:v main10 -pix_fmt yuv420p10le -vf scale=1280:720,"drawtext=fontfile=/content/drive/MyDrive/Font/A.ttf:text=Encoded By @Ani_Animesh:x=1040:y=650:fontsize=21:fontcolor=white:enable=between(t\,200\,440)" -preset medium -x265-params $x265_setting -crf $set_crf -map 0:0 -map $set_audio -map $set_audio1 -map 0:s -c:a aac -b:a 112k -metadata title="Encoded by Ani Animesh | AnimeSakura.co" -metadata:s:0 title="Episodes Unknown xD" -metadata:s:a:0 title="Ani Animesh" -metadata:s:a:1 title="AnimeSakura.co" -metadata:s:s:0 title="AnimeSakura.co" -metadata:s:s:1 title="@Ani_Animesh"  "$output_folder/{filename.rpartition('.')[0]}.$output_file_type"
if encode_setting == '1080p sub':
 for filename in os.listdir(input_folder):
     if (filename.endswith(my_suffixes)):
             cmd = !ffmpeg -stats -i "$input_folder/{filename}" -metadata comment="X265-CRF$set_crf-AAC" -metadata title="Encoded By Ani Animesh | AnimeToki.com" -metadata author="AnimeToki.com" -metadata copyright="Copyright 2025 AniEncode" -metadata encoded_by="Ani Animesh" -metadata:s:0 title="Encoded By Ani Animesh | AnimeToki.com" -metadata:s:a:0 title="Ani Animesh" -metadata:s:a:1 title="AnimeToki.Com" -metadata:s:s:0 title="AnimeToki.com" -metadata:s:s:1 title="@Ani_Animesh" -metadata:s:s:2 title="AnimeToki.com" -map 0 -codec copy  "$output_folder/[AnimeToki] {filename.rpartition('.')[0]} - 1080p English Subb.$output_file_type"
if encode_setting == 'X265 Copy':
 for filename in os.listdir(input_folder):
     if (filename.endswith(my_suffixes)):
             cmd = !ffmpeg -stats -i "$input_folder/{filename}" -metadata comment="X265-CRF$set_crf-AAC" -metadata title="Encoded By Ani Animesh | AnimeToki.com" -metadata author="AnimeToki.com" -metadata copyright="Copyright 2024 AniEncode" -metadata encoded_by="Ani Animesh" -metadata:s:0 title="Encoded By Ani Animesh | AnimeToki.com" -metadata:s:a:0 title="Ani Animesh" -metadata:s:a:1 title="AnimeToki.Com" -metadata:s:s:0 title="AnimeToki.com" -metadata:s:s:1 title="@Ani_Animesh" -metadata:s:s:2 title="AnimeToki.com" -map 0 -codec copy "$output_folder/{filename.rpartition('.')[0]}.$output_file_type"
if encode_setting == 'X265 AAC 1080p':
 for filename in os.listdir(input_folder):
     if (filename.endswith(my_suffixes)):
             cmd = !ffmpeg -stats -i "$input_folder/{filename}" -metadata comment="X265-CRF$set_crf-AAC" -c:v libx265 -color_primaries 1 -color_range 1 -color_trc 1 -colorspace 1 -profile:v main10 -pix_fmt yuv420p10le -preset medium -vf smartblur=1.5:-0.35:-3.5:0.65:0.25:2.0,scale=1920:1080:spline16+accurate_rnd+full_chroma_int -crf $set_crf -map 0:0 -map $set_audio -map $set_audio1 -map 0:s -c:a aac -b:a 192k -metadata title="Encoded By Ani Animesh" -metadata:s:0 title="Presented By Sakura Anime" -metadata:s:a:0 title="Ani Animesh" -metadata:s:a:1 title="Sakura Anime" -metadata:s:s:0 title="Sakura Anime" -metadata:s:s:1 title="@Animesh20" -metadata:s:s:2 title="@AnimeChannelSakura" "$output_folder/{filename.rpartition('.')[0]}.$output_file_type"
if encode_setting == '1080p dub':
  for filename in os.listdir(input_folder):
      if (filename.endswith(my_suffixes)):
            cmd = !ffmpeg -stats -i "$input_folder/{filename}" -codec copy -metadata comment="AniEcode" -metadata title="Encoded By Ani Animesh | AnimeToki.com" -metadata author="AnimeToki.com" -metadata copyright="Copyright 2025 AniEncode" -metadata encoded_by="Ani Animesh" -metadata:s:0 title="Encoded By Ani Animesh | AnimeToki.com" -metadata:s:a:0 title="Ani Animesh" -metadata:s:a:1 title="AnimeToki.Com" -metadata:s:s:0 title="AnimeToki.com" -metadata:s:s:1 title="@Ani_Animesh" -metadata:s:s:2 title="AnimeToki.com" -map 0 "$output_folder/[AnimeToki] {filename.rpartition('.')[0]} - 1080p Dual Audio Sub.$output_file_type"
if encode_setting == '720p sub':
 for filename in os.listdir(input_folder):
     if (filename.endswith(my_suffixes)):
             cmd = !ffmpeg -stats -i "$input_folder/{filename}" -metadata comment="AnimeToki.com" -metadata title="Encoded By Ani Animesh | AnimeToki.com" -metadata author="AnimeToki.com" -metadata copyright="Copyright 2025 AniEncode" -metadata encoded_by="Ani Animesh" -metadata:s:0 title="Encoded By Ani Animesh | AnimeToki.com" -metadata:s:a:0 title="Ani Animesh" -metadata:s:a:1 title="AnimeToki.Com" -metadata:s:s:0 title="AnimeToki.com" -metadata:s:s:1 title="@Ani_Animesh" -metadata:s:s:2 title="AnimeToki.com" -map 0 -codec copy  "$output_folder/[AnimeToki] {filename.rpartition('.')[0]} - 720p English Subb.$output_file_type"
if encode_setting == '720p dub':
 for filename in os.listdir(input_folder):
     if (filename.endswith(my_suffixes)):
             cmd = !ffmpeg -stats -i "$input_folder/{filename}" -metadata comment="AnimeToki.com" -metadata title="Encoded By Ani Animesh | AnimeToki.com" -metadata author="AnimeToki.com" -metadata copyright="Copyright 2025 AniEncode" -metadata encoded_by="Ani Animesh" -metadata:s:0 title="Encoded By Ani Animesh | AnimeToki.com" -metadata:s:a:0 title="Ani Animesh" -metadata:s:a:1 title="AnimeToki.Com" -metadata:s:s:0 title="AnimeToki.com" -metadata:s:s:1 title="@Ani_Animesh" -metadata:s:s:2 title="AnimeToki.com" -map 0 -codec copy  "$output_folder/[AnimeToki] {filename.rpartition('.')[0]} - 720p Dual Audio Subb.$output_file_type"

In [ ]:
#@markdown <h3><b>FFMPEG Encode SVTAV1 10bit :</h3>

import os, sys, re

input_folder= '' #@param {type:"string"}
output_folder= '' #@param {type:"string"}
#@markdown <h3><b>Choose your encode setting :</h5>
#@markdown <h><i>main = default | main10 = HEVC 10 Bit</i>
encode_setting = "AV1 AAC 10bit" #@param ["AV1 AAC 10bit"]
output_file_type = 'mkv' #@param ["mkv", "mp4"]
#@markdown <h3><b>Video quality :</h3>
#@markdown <h><i>0 (Best) to 51 (Worst)</i><h>
resolution = '1280:720' #@param ["1024:575", "1280:720", "1920:1080"]
set_preset ='8'#@param {type:"string"}
set_crf ='28'#@param {type:"string"}
set_audio ='0:1' #@param {type:"string"}

set_audio1 ='0:2' #@param {type:"string"}
my_suffixes = (".mp4", ".mov", ".mkv", ".avi", ".ts", ".flv", ".webm", ".wmv", ".mpg", ".m4v", ".f4v", ".m2ts", ".mpeg", ".3gp", ".MP4", ".MOV", ".MKV", ".AVI", ".TS", ".FLV", ".WEBM", ".WMV", ".MPG", ".M4V", ".F4V", ".M2TS", ".MPEG", ".3GP")

from pathlib import Path
Path(output_folder).mkdir(parents=True, exist_ok=True)

if encode_setting == 'AV1 AAC 10bit':
 for filename in os.listdir(input_folder):
     if (filename.endswith(my_suffixes)):
             cmd = !ffpb -i "$input_folder/{filename}" -metadata comment="SvtAv1-CRF$set_crf-AAC" -c:v libsvtav1 -preset $set_preset -g 240 -s $resolution -pix_fmt yuv420p10le -color_primaries 1 -color_range 1 -color_trc 1 -colorspace 1 -svtav1-params "tune=1:pass=0:film-grain=0:keyint=240:profile=0:lp=4:enable-hdr=1" -vf "drawtext=fontfile=/content/drive/MyDrive/Font/A.ttf:text=Encoded By @Ani_Animesh:x=1580:y=980:fontsize=25:fontcolor=white:enable=between(t\,200\,400)","drawtext=fontfile=/content/drive/MyDrive/Font/cream-DEMO.ttf:text=www.AnimeSakura.co:fontsize=20:fontcolor=white:x=1580:y=970:enable=between(t\,720\,750)" -crf $set_crf -map 0:v -map $set_audio -map $set_audio1 -map 0:s -c:a aac -b:a 96k -metadata title="Episode Unknow xD" -metadata author="AnimeSakura.co" -metadata copyright="Copyright 2023 Me" -metadata encoded_by="Ani Animesh" -metadata:s:0 title="Presented By Anime Sakura" -metadata:s:a:0 title="Ani Animesh" -metadata:s:a:1 title="AnimeSakura.co" -metadata:s:s:0 title="AnimeaSakura.co" -metadata:s:s:1 title="@Ani_Animesh"  "$output_folder/{filename.rpartition('.')[0]}.$output_file_type"




In [ ]:
#@markdown <h3><b>FFMPEG Encode x264 10bit :</h3>

import os, sys, re

input_folder= '' #@param {type:"string"}
output_folder= '' #@param {type:"string"}
#@markdown <h3><b>Choose your encode setting :</h5>
#@markdown <h><i>main = default | main10 = HEVC 10 Bit</i>
encode_setting = "X264 AAC 720p" #@param ["X264 AAC 720p", "X264 AAC 1080p"]
output_file_type = 'mkv' #@param ["mkv", "mp4"]
#@markdown <h3><b>Video quality :</h3>
#@markdown <h><i>0 (Best) to 51 (Worst)</i><h>
set_crf ='22.2'#@param {type:"string"}
set_audio ='0:1' #@param {type:"string"}

set_audio1 ='0:2' #@param {type:"string"}
my_suffixes = (".mp4", ".mov", ".mkv", ".avi", ".ts", ".flv", ".webm", ".wmv", ".mpg", ".m4v", ".f4v", ".m2ts", ".mpeg", ".3gp", ".MP4", ".MOV", ".MKV", ".AVI", ".TS", ".FLV", ".WEBM", ".WMV", ".MPG", ".M4V", ".F4V", ".M2TS", ".MPEG", ".3GP")

from pathlib import Path
Path(output_folder).mkdir(parents=True, exist_ok=True)

if encode_setting == 'X264 AAC 720p':
 for filename in os.listdir(input_folder):
     if (filename.endswith(my_suffixes)):
             cmd = !ffpb -stats  -i "$input_folder/{filename}" -metadata comment="X264-CRF$set_crf-opus" -c:v libx264 -color_primaries 1 -color_range 1 -color_trc 1 -colorspace 1 -profile:v high10 -pix_fmt yuv420p10le -level:v 4.1 -vf scale=1280:720,"drawtext=fontfile=/content/drive/MyDrive/Font/A.ttf:text=Encoded By @Ani_Animesh:x=1040:y=650:fontsize=20:fontcolor=white:enable=between(t\,420\,700)","drawtext=fontfile=/content/drive/MyDrive/Font/cream-DEMO.ttf:text=www.AnimeToki.com:fontsize=20:fontcolor=white:x=1040:y=650:enable=between(t\,720\,750)" -preset slower  -tune animation -x264-params me=umh:ref=6:deblock=1,1,1:subme=7:psy=1:psy_rd=0.40:me_range=16:chroma_me=1:trellis=2:fast_pskip=1:chroma_qp_offset=-2:threads=18:lookahead_threads=4:sliced_threads=0:nr=0:interlaced=0:bluray_compat=1:constrained_intra=0:bframes=5:b_pyramid=2:b_adapt=2:b_bias=0:direct=1:weightb=1:open_gop=0:scenecut=40:intra_refresh=0:rc_lookahead=40:qcomp=0.60:qpmin=10:qpmax=51:qpstep=4 -crf $set_crf -map 0:0 -map $set_audio -map $set_audio1 -map 0:s -c:a libopus -b:a 112k -metadata title="Episode Unknow xD" -metadata author="AnimeToki.com" -metadata copyright="Copyright 2025 Me" -metadata:s:0 title="Presented By AnimeToki" -metadata:s:a:0 title="Ani Animesh" -metadata:s:a:1 title="AnimeToki.com" -metadata:s:s:0 title="AnimeToki" -metadata:s:s:1 title="@Ani_Animesh"  "$output_folder/{filename.rpartition('.')[0]}.$output_file_type"
if encode_setting == 'X264 AAC 1080p':
 for filename in os.listdir(input_folder):
     if (filename.endswith(my_suffixes)):
             cmd = !ffmpeg -stats -i "$input_folder/{filename}" -metadata comment="X265-CRF$set_crf-AAC" -c:v libx264 -color_primaries 1 -color_range 1 -color_trc 1 -colorspace 1 -profile:v high10 -pix_fmt yuv420p10le -level:v 5.2 -vf scale=1920:1080,"drawtext=fontfile=/content/drive/MyDrive/Font/A.ttf:text=Encoded By @Ani_Animesh:x=1680:y=980:fontsize=20:fontcolor=white:enable=between(t\,220\,500)","drawtext=fontfile=/content/drive/MyDrive/Font/cream-DEMO.ttf:text=www.AnimeToki.com:fontsize=25:fontcolor=white:x=1680:y=980:enable=between(t\,720\,750)" -preset medium  -tune animation -x264-params me=umh:ref=6:deblock=1,1,1:subme=7:psy=1:psy_rd=0.40:me_range=16:chroma_me=1:trellis=2:fast_pskip=1:chroma_qp_offset=-2:threads=18:lookahead_threads=4:sliced_threads=0:nr=0:interlaced=0:bluray_compat=0:constrained_intra=0:bframes=5:b_pyramid=2:b_adapt=2:b_bias=0:direct=1:weightb=1:open_gop=0:scenecut=40:intra_refresh=0:rc_lookahead=40:qcomp=0.60:qpmin=10:qpmax=51:qpstep=4 -crf $set_crf -map 0:0 -map $set_audio -map $set_audio1 -map 0:s -c:a aac -b:a 128k -metadata title="Episode Unknow xD" -metadata author="AnimeToki.com" -metadata copyright="Copyright 2025 Me" -metadata:s:0 title="Presented By AnimeToki" -metadata:s:a:0 title="Ani Animesh" -metadata:s:a:1 title="AnimeToki.com" -metadata:s:s:0 title="AnimeToki" -metadata:s:s:1 title="@Ani_Animesh" "$output_folder/{filename.rpartition('.')[0]}.$output_file_type"

In [ ]:
import os, sys, re

video_file_path = "" #@param {type:"string"}
output_file_extension = 'm4a' #@param ["m4a", "aac", "opus", "flac", "wav"]

delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
output_file_path = re.search("^[\/].+\/", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path.group(0)
os.environ['fileName'] = filename_raw
os.environ['fileType'] = output_file_extension

!ffpb -y -hide_banner -i "$inputFile" -vn -c:a copy "$outputPath"/"$fileName"-audio."$fileType"

In [ ]:
import os, sys, re

video_file_path = "" #@param {type:"string"}
audio_file_path = "" #@param {type:"string"}

output_file_path = re.search("^[\/].+\/", video_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = video_file_path
os.environ['inputAudioFile'] = audio_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

!ffmpeg -hide_banner -i "$inputFile" -i "$inputAudioFile" -c:v copy -map 0:0 -map 1 -map 0:a -map 0:s -c:a aac -b:a 112k -disposition:a:0 default -metadata title="Encoded By Ani Animesh" -metadata:s:0 title="Presented By Anime Sakura" -metadata:s:a:0 title="AnimeSakura.co" -metadata:s:a:1 title="Ani Animesh" -metadata:s:s:0 title="AnimeSakura.co" -metadata:s:s:1 title="@Ani_Animesh" -metadata author="AnimeSakura.co" -metadata encoded_by="Lolikiller #Sakura_gang" -metadata:s:a:0 language="eng" "$outputPath"/"$fileName"-out.mkv

In [ ]:
#@title <font size="3"> *You can ignore the* `At least one output file must be specified` *error for this cell*
import os, sys, re
media_file_path = "" #@param {type:"string"}
os.environ['inputFile'] = media_file_path
!ffmpeg -i "$inputFile" -hide_banner

In [ ]:
#@title <<----<strong>Install HandBrake and rClone</strong>
%%capture
AUTO_RECONNECT = True #@param {type:"boolean"}
HANDBRAKE = True #@param {type:"boolean"}
RCLONE = False #@param {type:"boolean"}
#@markdown Check AUTO_RECONNECT to prevent notebook from disconnecting!

from os import makedirs
makedirs("/content/temp/HandbrakeTemp", exist_ok = True)
makedirs("/root/.config/rclone", exist_ok = True)
if HANDBRAKE==True:
  !wget -qq https://github.com/vot/ffbinaries-prebuilt/releases/download/v4.2.1/ffmpeg-4.2.1-linux-64.zip
  !rm -f ffmpeg-4.2.1-linux-64.zip
  !add-apt-repository ppa:stebbins/handbrake-releases -y
  !apt-get install -y handbrake-cli

if RCLONE==True:
  !curl https://rclone.org/install.sh | sudo bash

if AUTO_RECONNECT:
  import IPython
  from google.colab import output

  display(IPython.display.Javascript('''
  function ClickConnect(){
    btn = document.querySelector("colab-connect-button")
    if (btn != null){
      console.log("Click colab-connect-button");
      btn.click()
      }

    btn = document.getElementById('ok')
    if (btn != null){
      console.log("Click reconnect");
      btn.click()
      }
    }

  setInterval(ClickConnect,60000)
  '''))

In [ ]:
#@title <center> HandBrake Configuration </center>
#@markdown > Select Mode ( Batch conversion/ Single File)
MODE = "SINGLE" #@param ["SINGLE", "BATCH"]


#@markdown ---
SOURCE = "" #@param {type:"string"}
DESTINATION = "" #@param {type:"string"}
FORMAT = "mkv" #@param ["mp4", "mkv"]
RESOLUTION = "720p" #@param ["360p", "480p","540p", "720p", "1080p"]
Encoder = "x265_10bit" #@param ["x264_10bit", "x265_10bit"]
Encoder_Preset = "medium" #@param ["ultrafast", "faster", "fast", "medium", "slow", "slower"]
#@markdown Choose Constant Quality Rate [Lower = Higher Quality/
#@markdown Larger File Size]
CQ = 21 #@param {type:"slider", min:10, max:30, step:1}
BURN_SUBTITLES = False #@param {type:"boolean"}
Additional_Flags = "" #@param {type:"string"}
#@markdown ---
Email_Alert = False #@param {type:"boolean"}
MESSAGE = "HandBrakeCLI_Colab has finished working" #@param ["HandBrake has finished working"] {allow-input: true}
SENDER_ID ="" #@param {type:"string"}
SENDER_PASS="" #@param {type:"string"}
RECEIVER_ID="" #@param {type:"string"}
#@markdown >#####  Using an app specific password is recommended: https://myaccount.google.com/apppasswords

########################################################
import smtplib
import os


formats = ('.mkv','.mp4','.ts','.avi','.mov','.wmv')

######## Renames the file ########
def fileName(fPath):
        tName = fPath.split('/')[-1]
        if tName.endswith('ts'):
          tName = '[ANI] ' + tName[:-3] + f' [{RESOLUTION}] [{Encoder}].{FORMAT}'
        else:
          tName = '[ANI] ' + tName[:-4] + f' [{RESOLUTION}] [{Encoder}].{FORMAT}'
        return tName

def set_resolution():
  global w,h,flags
  if RESOLUTION == "360p":
    w, h = "480" , "360"
  elif RESOLUTION == "480p":
    w, h = "854" , "480"
  elif RESOLUTION == "540p":
    w, h = "960" , "540"
  elif RESOLUTION == "720p":
    w, h = "1280" , "720"
  elif RESOLUTION == "1080p":
    w, h = "1920" , "1080"

def addFlags():
  global flags
  flags = f" --encoder {Encoder} -a '1,2' -M 2020 --aencoder 'aac' -B '96kbps' --aname 'Ani Animesh,Anime Sakura' -s 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20 --cfr  --quality={CQ} --width={w} --height={h} --format={FORMAT} --encoder-preset={Encoder_Preset} "
  if BURN_SUBTITLES:
    flags += "-s '1' --subtitle-burned '1' "
  if Additional_Flags != "":
    flags += str(Additional_Flags)

set_resolution()
addFlags()

##### HandBrake and Rclone #####
def runner(path):
  f_name = fileName(path)
  hTemp=f"/content/temp/HandbrakeTemp/{f_name}"
  !HandBrakeCLI -i "$path" -o "$hTemp" $flags


  if os.path.isfile(hTemp):
    print(f"\n\n********** Successfully converted {f_name}\n Now saving to Destination.....")
    if os.path.exists('/usr/bin/rclone'):
      !rclone move "$hTemp" --user-agent "Mozilla" "$DESTINATION" --transfers 20 --checkers 20 --stats-one-line --stats=5s -v --tpslimit 95 --tpslimit-burst 40
    else:
      dest = DESTINATION+'/'+f_name
      !mv "$hTemp" "$dest"
  if os.path.isfile(DESTINATION+ '/' +f_name):
      print(f"\n\n********** Successfully saved {f_name} to Destination")

########## Check Mode ########
if MODE=="BATCH":
  os.makedirs(DESTINATION, exist_ok=True)
  if SOURCE.endswith('/'):
      pass
  else: SOURCE +='/'
  filesList = os.listdir(SOURCE+'.')
  if os.path.isfile(SOURCE+'processed_db.txt'):
    pass
  else:
    with open((SOURCE+'processed_db.txt'), 'w') as fb:
      fb.write("Do not delete this file until all files have been processed!\n")
      fb.close()
  with open((SOURCE+'processed_db.txt'), "r+") as filehandle:
    processedList = [x.rstrip() for x in filehandle.readlines()]

    print('<<<<<<<<<<<<<<<<<< Starting Conversion in Batch mode. >>>>>>>>>>>>>>>>>>')

    for currentFile in filesList:
      if currentFile.endswith(formats):
        if currentFile not in processedList:
          currentPath = SOURCE + currentFile
          print(f'\n\n**************** Current File to process: {currentFile}')
          runner(currentPath)
          filehandle.write(currentFile+'\n')
    filehandle.close()


else:
    if SOURCE.endswith(formats):
        runner(SOURCE)
    else: print("Are you sure you have selected the correct file??")

########### Sending Notification ###############
if Email_Alert == True:
  from email.mime.multipart import MIMEMultipart
  from email.mime.text import MIMEText
  msg = MIMEMultipart()
  msg['From'] = SENDER_ID
  msg['To'] = RECEIVER_ID
  msg['Subject'] = "HandBrakeCLI_Colab has finished executing!"

  msg.attach(MIMEText(MESSAGE))
  s = smtplib.SMTP('smtp.gmail.com', 587)
  s.starttls()
  s.login(SENDER_ID, SENDER_PASS)
  s.sendmail(SENDER_ID, RECEIVER_ID, msg.as_string())
  print("Email Alert Sent!")
  s.quit()
